In [19]:
# LIBRERÍAS
import pandas as pd
import matplotlib.pyplot as plt

In [24]:
pd.__version__

'2.2.2'

In [10]:
# CLASES

class IPVDataLoader:
    """Clase para cargar y limpiar los datos del archivo CSV."""
    def __init__(self, file_path):
        self.file_path = file_path

    def load_data(self):
        df = pd.read_csv(self.file_path, sep='\t', encoding='utf-8')
        df['Total'] = df['Total'].str.replace(',', '.').astype(float)
        df = df.dropna(subset=['Total', 'Periodo'])
        df = df.reset_index(drop=True)

        return df

In [26]:
class ComunidadIPV:
    """Clase que representa una comunidad autónoma y calcula métricas."""
    def __init__(self, nombre, datos):
        self.nombre = nombre
        self.datos = datos.sort_values('Periodo')

    def indice_actual(self):
        return self.datos.iloc[-1]['Total']

    def variacion_anual(self):
        if len(self.datos) < 5:
            return None
        return ((self.datos.iloc[-1]['Total'] / self.datos.iloc[-5]['Total']) - 1) * 100

    def variacion_trimestral(self):
        if len(self.datos) < 2:
            return None
        return ((self.datos.iloc[-1]['Total'] / self.datos.iloc[-2]['Total']) - 1) * 100

    def variacion_desde_2015(self):
        base = self.datos[self.datos['Periodo'].str.contains('2015')]
        if not base.empty:
            return ((self.datos.iloc[-1]['Total'] / base.iloc[0]['Total']) - 1) * 100
        else:
            return None

    def calcular_metricas(self):
        return {
            'Comunidad': self.nombre,
            'Índice Actual': self.indice_actual(),
            'Variación Anual (%)': self.variacion_anual(),
            'Variación Trimestral (%)': self.variacion_trimestral(),
            'Variación Desde 2015 (%)': self.variacion_desde_2015()
        }

In [12]:
class IPVAnalyzer:
    """Clase que analiza todas las comunidades."""
    def __init__(self, df):
        self.df = df
        self.comunidades = self._crear_comunidades()

    def _crear_comunidades(self):
        comunidades = {}
        df_com = self.df[self.df['Comunidades y Ciudades Autónomas'].notna()]
        for comunidad in df_com['Comunidades y Ciudades Autónomas'].unique():
            datos_com = df_com[df_com['Comunidades y Ciudades Autónomas'] == comunidad]
            comunidades[comunidad] = ComunidadIPV(comunidad, datos_com)
        return comunidades

    def resumen_total(self):
        metricas = [comunidad.calcular_metricas() for comunidad in self.comunidades.values()]
        return pd.DataFrame(metricas).sort_values(by='Índice Actual', ascending=False)

    def ranking_crecimiento(self):
        resumen = self.resumen_total()
        return resumen.sort_values(by='Variación Anual (%)', ascending=False)